In [1]:
import pandas as pd
import json
import glob
from pandas.io.json import json_normalize

pd.options.display.max_rows = 999
pd.options.display.max_columns = 200
from IPython.display import HTML
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
df_concat = pd.read_pickle(r'.\data\raw_data.pkl')
df_concat.head()

,contract_date,contract_finish_date,contract_no,contract_status,corporate_name,price_agree,winner_tin,announce_date,budget_year,dept_name,dept_sub_name,district,price_build,project_id,project_money,project_name,project_status,project_type_name,province,purchase_method_name,subdistrict,sum_price_agree,transaction_date
0,8 ก.พ. 61,9 เม.ย. 61,3/2561,ระหว่างดำเนินการ,บริษัท สุขุมการโยธา จำกัด,"8,000,000",0315536000133,4 ธ.ค. 60,2561,องค์การบริหารส่วนจังหวัดตราด,องค์การบริหารส่วนจังหวัดตราด,เมืองตราด,"9,420,000",60117271655,"8,849,400.00",ประกวดราคาจ้างก่อสร้างปรับปรุงซ่อมแซมถนนลาดยาง...,ระหว่างดำเนินการ,จ้างก่อสร้าง,ตราด,ประกวดราคาอิเล็กทรอนิกส์ (e-bidding),วังกระแจะ,"8,000,000",8 ก.พ. 61
1,8 ก.พ. 61,9 เม.ย. 61,4/2561,ระหว่างดำเนินการ,บริษัท สุขุมการโยธา จำกัด,"7,950,000",0315536000133,4 ธ.ค. 60,2561,องค์การบริหารส่วนจังหวัดตราด,องค์การบริหารส่วนจังหวัดตราด,เมืองตราด,"9,360,000",60117286715,"9,163,000.00",ประกวดราคาจ้างก่อสร้างปรับปรุงซ่อมแซมถนนแอสฟัล...,ระหว่างดำเนินการ,จ้างก่อสร้าง,ตราด,ประกวดราคาอิเล็กทรอนิกส์ (e-bidding),วังกระแจะ,"7,950,000",8 ก.พ. 61
2,13 มี.ค. 61,9 ต.ค. 61,7/2561,ระหว่างดำเนินการ,บริษัท เค ดีเทล จำกัด,"7,890,000",0225559000254,-,2561,องค์การบริหารส่วนจังหวัดตราด,องค์การบริหารส่วนจังหวัดตราด,เมืองตราด,"8,571,480",61027021899,"7,891,300.00",จ้างก่อสร้างก่อสร้างอาคารเรียน 3 ชั้น 12 ห้องเ...,ระหว่างดำเนินการ,จ้างก่อสร้าง,ตราด,คัดเลือก,วังกระแจะ,"7,890,000",13 มี.ค. 61
3,24 ก.ย. 61,23 ธ.ค. 61,15/2561,ระหว่างดำเนินการ,ห้างหุันส่วนจำกัด เอส ซี เอวี อินเตอร์เนชั่นแนล,"4,800,000",0103550028503,5 ก.ย. 61,2561,องค์การบริหารส่วนจังหวัดตราด,องค์การบริหารส่วนจังหวัดตราด,เมืองตราด,"4,930,000",61097035003,"5,000,000.00",ประกวดราคาจ้างก่อสร้างติดตั้งระบบเสียงภายใน หอ...,ระหว่างดำเนินการ,จ้างก่อสร้าง,ตราด,ประกวดราคาอิเล็กทรอนิกส์ (e-bidding),วังกระแจะ,"4,800,000",24 ก.ย. 61
4,6 มิ.ย. 61,4 ก.ย. 61,12/2561,ระหว่างดำเนินการ,บริษัท เอ็น.พี.ซี.การโยธา จำกัด,"3,135,000",0225533000290,4 พ.ค. 61,2561,องค์การบริหารส่วนจังหวัดตราด,องค์การบริหารส่วนจังหวัดตราด,เมืองตราด,"3,350,000",61057023597,"3,300,000.00",ประกวดราคาจ้างก่อสร้างปรับปรุงถนนลาดยางแอสฟัลท...,ระหว่างดำเนินการ,จ้างก่อสร้าง,ตราด,ประกวดราคาอิเล็กทรอนิกส์ (e-bidding),วังกระแจะ,"3,135,000",6 มิ.ย. 61


In [3]:
df_concat['project_money'] = df_concat['project_money'].apply(lambda x: float(str(x).replace(',','')))
df_concat['price_agree'] = df_concat['price_agree'].apply(lambda x: float(str(x).replace(',','')))
df_concat['sum_price_agree'] = df_concat['sum_price_agree'].apply(lambda x: float(str(x).replace(',','')))

In [4]:
def civil_proj(x):
    include_key_words = ['ลูกรัง','ลาดยาง','สร้างถนน','ซ่อมแซมถนน','ซ่อมผิวจราจร','ท่อระบายน้ำ','หินคลุก','คสล',\
                 'คอนกรีตเสริมเหล็ก','ซ่อมแซมถนน','แอสฟัล','ประปา','บาดาล','บาดาล','สะพาน','ตลิ่ง']
    exclude_key_words = ['จ้างเหมาบุคคล','จ้างเหมาบริการ']
    
    bool_result = False
    
    for a_key_word in include_key_words:
        bool_result |= (a_key_word in x)
    
    for a_key_word in exclude_key_words:
        if a_key_word in x:
            return False

    return bool_result

In [6]:
df_concat['civil_proj_f']=df_concat['project_name'].apply(lambda x: civil_proj(x))
df_concat[df_concat['civil_proj_f']][['corporate_name','dept_name','project_name']]

,corporate_name,dept_name,project_name
0,บริษัท สุขุมการโยธา จำกัด,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างปรับปรุงซ่อมแซมถนนลาดยาง...
1,บริษัท สุขุมการโยธา จำกัด,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างปรับปรุงซ่อมแซมถนนแอสฟัล...
4,บริษัท เอ็น.พี.ซี.การโยธา จำกัด,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างปรับปรุงถนนลาดยางแอสฟัลท...
6,ห้างหุ้นส่วนจำกัด เทพวิไล,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างหินเรียงยาแนวกั้นน้ำเซาะ...
7,ห้างหุ้นส่วนจำกัด เทพวิไลคอนกรีต,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างก่อสร้างถนนคอนกรีตเสริมเ...
8,ห้างหุ้นส่วนจำกัด จีรนารถก่อสร้าง,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างปรับปรุงถนนลาดยางแบบผิวเ...
9,ห้างหุ้นส่วนจำกัด ตราดคอนกรีต,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างถนนลาดยางแอสฟัลท์ติกคอนก...
10,ห้างหุ้นส่วนจำกัด เกาะกูดวัสดุการโยธา,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างก่อสร้างถนนคอนกรีตเสริมเ...
11,ห้างหุ้นส่วนจำกัด เทพวิไลคอนกรีต,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างปรับปรุงต่อเติมเส้นทางเด...
13,ห้างหุ้นส่วนจำกัด ตราดคอนกรีต,องค์การบริหารส่วนจังหวัดตราด,ประกวดราคาจ้างก่อสร้างถนนคอนกรีตเสริมเหล็ก ซอย...
